# Compare NLP Techniques: Build Model On doc2vec Vectors

### Read In Cleaned Text

In [1]:
# Load the cleaned training and test sets
import gensim
import numpy as np
import pandas as pd

X_train = pd.read_csv('../../../data/X_train.csv')
X_test = pd.read_csv('../../../data/X_test.csv')
y_train = pd.read_csv('../../../data/y_train.csv')
y_test = pd.read_csv('../../../data/y_test.csv')

### Create doc2vec Vectors

In [2]:
# Created TaggedDocument vectors for each text message in the training and test sets
tagged_docs_train = [gensim.models.doc2vec.TaggedDocument(v, [i])
                     for i, v in enumerate(X_train['clean_text'])]
tagged_docs_test = [gensim.models.doc2vec.TaggedDocument(v, [i])
                    for i, v in enumerate(X_test['clean_text'])]

In [3]:
# What do these TaggedDocument objects look like?
tagged_docs_train[:10]

[TaggedDocument(words="['living', 'simple', 'loving', 'also', 'simple', 'laughing', 'simple', 'winning', 'tooo', 'simple', 'simple', 'difficult', '']", tags=[0]),
 TaggedDocument(words="['already', 'squatting', 'new', 'way', 'walking']", tags=[1]),
 TaggedDocument(words="['sister', 'got', 'placed', 'birla', 'soft', 'da']", tags=[2]),
 TaggedDocument(words="['lovely', 'smell', 'bus', 'aint', 'tobacco', '']", tags=[3]),
 TaggedDocument(words="['yes', 'nigh', 'cant', 'aha']", tags=[4]),
 TaggedDocument(words="['yup', 'remb', 'think', 'book', '']", tags=[5]),
 TaggedDocument(words="['sorry', 'roommates', 'took', 'forever', 'ok', 'come']", tags=[6]),
 TaggedDocument(words="['b4u', 'voucher', 'wc', '2703', 'marsms', 'log', 'onto', 'wwwb4utelecom', 'discount', 'credit', 'opt', 'reply', 'stop', 'customer', 'care', 'call', '08717168528']", tags=[7]),
 TaggedDocument(words="['kindly', 'send', 'one', 'flat', 'ltdecimalgt', 'today']", tags=[8]),
 TaggedDocument(words="['ceri', 'u', 'rebel', 'sweet

In [4]:
# Train a basic doc2vec model
d2v_model = gensim.models.Doc2Vec(tagged_docs_train,
                                  vector_size=100,
                                  window=5,
                                  min_count=2)

In [5]:
# Infer the vectors to be used in training and testing
train_vectors = [d2v_model.infer_vector(eval(v.words)) for v in tagged_docs_train]
test_vectors = [d2v_model.infer_vector(eval(v.words)) for v in tagged_docs_test]   

### Fit RandomForestClassifier On Top Of Document Vectors

In [6]:
# Fit a basic model, make predictions on the holdout test set, and the generate the evaluation metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score

rf = RandomForestClassifier()
rf_model = rf.fit(train_vectors, y_train.values.ravel())

y_pred = rf_model.predict(test_vectors)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test['label']).sum()/len(y_pred), 3)))

Precision: 0.771 / Recall: 0.367 / Accuracy: 0.902
